In [1]:
import pandas as pd

## 1. Đọc dữ liệu

In [2]:
df = pd.read_csv(
    'data/household_power_consumption.txt',
    sep=';',
    low_memory=False,
    na_values=['nan', '?']
)

df['dt'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)
df.set_index('dt', inplace=True)

In [3]:
df.head(10)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
2006-12-16 17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0
2006-12-16 17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0
2006-12-16 17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0
2006-12-16 17:32:00,3.668,0.510,233.99,15.8,0.0,1.0,17.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [5]:
df.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

## 2 Xử lý giá trị thiếu bằng nội suy tuyến tính

In [ ]:
df.interpolate(method='time', inplace=True)

In [7]:
df.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [8]:
df.shape

(2075259, 7)

In [9]:
# df = df.iloc[:500000, :]
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# import matplotlib.pyplot as plt

# fig, axes = plt.subplots(2, 1, figsize=(10, 6))
# plot_acf(df['Global_active_power'], lags=200, ax=axes[0])
# axes[0].set_title('Autocorrelation (ACF)')
# plot_pacf(df['Global_active_power'], lags=200, ax=axes[1])
# axes[1].set_title('Partial Autocorrelation (PACF)')

# # main title
# fig.suptitle('Global Active Power: ACF & PACF Analysis', fontsize=16)

# # adjust layout so suptitle isn’t cut off
# fig.tight_layout(rect=[0, 0.03, 1, 0.95])

# # save to file
# fig.savefig('acf_pacf_with_title.png', dpi=300, bbox_inches='tight')

# plt.show()

In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar

def preprocess_df(df: pd.DataFrame,
                  target_col: str = "Global_active_power",
                  dt_col: str = "dt") -> pd.DataFrame:
    """
    Tiền xử lý time series và sinh các feature:
      - Đặt dt_col thành DatetimeIndex, sort theo time
      - Fourier features: sin_hour, cos_hour, sin_dow, cos_dow
      - is_holiday (US Federal Holidays)
      - lag1, lag24, lag168
      - roll_mean_3h (mean trong 3 giờ)
      - Nội suy tuyến tính theo time, sau đó ffill + bfill để loại NaN
    """
    df_proc = df.copy()

    # --- Thiết lập index datetime ---
    if dt_col in df_proc.columns:
        df_proc[dt_col] = pd.to_datetime(df_proc[dt_col])
        df_proc.set_index(dt_col, inplace=True)
    else:
        df_proc.index = pd.to_datetime(df_proc.index)
    df_proc.sort_index(inplace=True)

    # --- Fourier (hour & day-of-week) ---
    df_proc['sin_hour'] = np.sin(2 * np.pi * df_proc.index.hour / 24)
    df_proc['cos_hour'] = np.cos(2 * np.pi * df_proc.index.hour / 24)
    df_proc['sin_dow']  = np.sin(2 * np.pi * df_proc.index.dayofweek / 7)
    df_proc['cos_dow']  = np.cos(2 * np.pi * df_proc.index.dayofweek / 7)

    # --- Holiday flag (US Federal) ---
    cal = USFederalHolidayCalendar()
    holidays_us = cal.holidays(start=df_proc.index.min(),
                               end=df_proc.index.max())
    df_proc['is_holiday'] = df_proc.index.normalize().isin(holidays_us).astype(int)

    # --- Lag features ---
    df_proc['lag1']   = df_proc[target_col].shift(1)
    df_proc['lag24']  = df_proc[target_col].shift(24)
    df_proc['lag168'] = df_proc[target_col].shift(168)

    # --- Rolling mean 3 giờ ---
    df_proc['roll_mean_3h'] = (
        df_proc[target_col]
          .rolling(window=3, min_periods=3)  # cần đủ 3 quan sát
          .mean()
    )

    # --- Nội suy tuyến tính theo time ---
    df_proc.interpolate(method='time', inplace=True)

    # --- Điền forward-fill và back-fill cho các NaN còn lại ---
    df_proc.ffill(inplace=True)
    df_proc.bfill(inplace=True)

    return df_proc

In [11]:
df_new = preprocess_df(df, target_col="Global_active_power")

In [15]:
df_new.head(10)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sin_hour,cos_hour,sin_dow,cos_dow,is_holiday,lag1,lag24,lag168,roll_mean_3h
dt,,,,,,,,,,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,-0.965926,-0.258819,-0.974928,-0.222521,0,4.216,4.216,4.216,4.983333
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,-0.965926,-0.258819,-0.974928,-0.222521,0,4.216,4.216,4.216,4.983333
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,-0.965926,-0.258819,-0.974928,-0.222521,0,5.360,4.216,4.216,4.983333
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,-0.965926,-0.258819,-0.974928,-0.222521,0,5.374,4.216,4.216,5.374000
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,-0.965926,-0.258819,-0.974928,-0.222521,0,5.388,4.216,4.216,4.809333
2006-12-16 17:29:00,3.520,0.522,235.02,15.0,0.0,2.0,17.0,-0.965926,-0.258819,-0.974928,-0.222521,0,3.666,4.216,4.216,4.191333
2006-12-16 17:30:00,3.702,0.520,235.09,15.8,0.0,1.0,17.0,-0.965926,-0.258819,-0.974928,-0.222521,0,3.520,4.216,4.216,3.629333
2006-12-16 17:31:00,3.700,0.520,235.22,15.8,0.0,1.0,17.0,-0.965926,-0.258819,-0.974928,-0.222521,0,3.702,4.216,4.216,3.640667
2006-12-16 17:32:00,3.668,0.510,233.99,15.8,0.0,1.0,17.0,-0.965926,-0.258819,-0.974928,-0.222521,0,3.700,4.216,4.216,3.690000


In [13]:
df_new.shape

(2075259, 16)

In [14]:
df_new.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
sin_hour                 0
cos_hour                 0
sin_dow                  0
cos_dow                  0
is_holiday               0
lag1                     0
lag24                    0
lag168                   0
roll_mean_3h             0
dtype: int64

In [16]:
df_new.to_csv('data/clean.csv', index=True)